# Generate parameters for HCP pipeline

### Objective:
For each subject, read the BIDS JSON-files to extract the parameters required by the pipeline.

The HCP Pipeline is used for structural and functional processing (incl.: PreFreeSurfer, FreeSurfer, PostFreeSurfer, fMRI Volume, fMRI Surface, ICAFIX, PostFIX, MSMAll, DedriftResample). Remaining processing done via non-HCP scripts (incl.:diffusion, PET).

For each step in the pipeline, we want batch job submitter scripts, containing "sbatch ..." commands for each subject for that step. The sbatch commands call "generic" wrapper scripts (for single subjects). These generic scripts, in turn, call the "actual" scripts (e.g. HCP pipeline scripts).

The number in the batch submitter filenames indicate the order in which they are to be executed.

### Prerequisites:
1. Data organized according to BIDS-specification (flattened if subject has data from multiple sessions)
2. Text file(s):
 * subject list
 * path list (only required in case subject has data from multiple sessions)

### Notes:

1. Current dataset does not include scans from Philips/GE scanners, and so skips these scans without handling them.
2. This script assumes you have the raw data locally, and you will be running the processing on a remote machine or supercomputer. The script therefore takes both "remote path" and "local path" names. If your data is already on the remote machine, you coulde e.g. set both these variables to the same (remote) path.

# Instructions:
1. Edit paths in cells: SUBLIST, PATHLIST, DEFINE PATHS  
2. import required packages
3. execute cells at bottom of script, to define the required functions
4. if data looks like
 * sub-1234/{ADNIVisit1,ADNIVisit2}/{anat, dwi, fmap, func, pet}: run everything unchanged.
 * sub-1234/{anat, dwi, fmap, func, pet}: skip "pathList" cell, and loop over subList (instead of pathList) in main script.

In [ ]:
import json
import pandas as pd
import glob
import os
import pydicom as dc
from datetime import datetime
import numpy as np
import re

SUBLIST

In [ ]:
# Read in subject list from preexisting ".txt" file, with each subject listed on a different line.
#subList.txt contains only entries of the form "sub-1234", not full path.
subList = open("/path/to/subList.txt","r").readlines()
subList = [x.strip("\n") for x in subList]

PATHLIST

In [ ]:
#if data is longitudinal (multiple visits per subject), e.g.: .../raw/sub-1234/{ADNIVisit1,ADNIVisit2}/{anat, dwi, fmap, func, pet}, the data needs to be flattened.
# This is so the HCP scripts handle the input files/create output files as expected.
# For each sub, for each visit, create a directory named, e.g.: sub-1234_ses-ADNIVisitX

#pathList.txt contains full paths for each session, e.g.: /..../raw/sub-1234/ADNIVisit1


pathList = open("/path/to/pathList.txt","r").readlines()
pathList = [x.strip("\n") for x in pathList]

for i,r in enumerate(pathList):
    groups = r.split('/')
    pathList[i] = '_ses-'.join(groups[-2:])

#print(len(pathList))
#print(len(subList))

# Main

DEFINE PATHS

In [ ]:
# "raw" data is a directory containing NIFTI files, organized according to the BIDS specification: $raw_data_path/$sub/$visit/{anat, dwi, fmap, func, pet}/***{.nii.gz, .json}
rawdata_local_path = "/path/to/raw"

#path on BIH cluster: raw data & results
rawdata_remote_path = "/path/to/raw/on/remoteComputer"
results_path = "/path/to/remote/resultsDir"

scratchPath="/path/to/remote/scratchDir" #directory for MRtrix to create large tck files

# local path: where to write the sbatch submitters
scripts_folder = "/path/to/write/batchSubmitterScripts"

FIX_training_file="/path/to/*_Training.RData"

WRITE BATCH SUBMITTER FILES

In [ ]:
# write the sbatch command to these files for each subject
batch_prefreesurf_list                 = open(scripts_folder+"/batch01_prefreesurf.sh","w")
batch_freesurf_list                    = open(scripts_folder+"/batch02_freesurf.sh","w")
batch_postfreesurf_list                = open(scripts_folder+"/batch03_postfreesurf.sh","w")
batch_fmri_volume_list                 = open(scripts_folder+"/batch04_fmri_volume.sh","w")
batch_fmri_surface_list                = open(scripts_folder+"/batch05_fmri_surface.sh","w")
batch_icafix_list                      = open(scripts_folder+"/batch06_icafix.sh","w")
batch_postfix_list                     = open(scripts_folder+"/batch07_postfix.sh","w")
batch_msmall_list                      = open(scripts_folder+"/batch08_msmall.sh","w")
batch_dedriftresample_list             = open(scripts_folder+"/batch09_dedriftresample.sh","w")
batch_denoise_preproc_list             = open(scripts_folder+"/batch10_denoise_preproc.sh","w")
batch_dwibiascorrect_dwi2mask_list     = open(scripts_folder+"/batch11_dwi_distortionCorrection.sh","w")
#batch_dwiintensitynorm_list            = open(scripts_folder+"/batch12_dwiintensitynorm.sh","w")             #only one line: uses all subs. write outside loop.
batch_T1w2dwi_5ttgen_dwi2response_list = open(scripts_folder+"/batch13_T1w2dwi_5ttgen_dwi2response.sh","w")
#batch_averageresponse_list            = open(scripts_folder+"/batch14_averageresponse.sh","w")             #only one line: uses all subs. write outside loop.
batch_dwi_2fod_tckgen_sift2_list       = open(scripts_folder+"/batch15_dwi_2fod_tckgen_sift2.sh","w")
batch_create_diffusion_mask_list       = open(scripts_folder+"/batch16_create_diffusion_mask.sh","w")
batch_tck2connectome_list              = open(scripts_folder+"/batch17_tck2connectome.sh","w")
batch_extract_PET_data_list            = open(scripts_folder+"/batch18_extract_PET_data.sh","w")
batch_extract_fmri_list                = open(scripts_folder+"/batch19_extract_fmri.sh","w")

for sub in pathList:
    
    ########################
    ##### Structural #######
    ########################

    # get parameters
    params_struct = structural_scan_params(rawdata_remote_path, rawdata_local_path, sub)
    if params_struct == False: continue # i.e. if Scanner Manufacturer == Philips/GE or unlisted

    # print to files
    batch_prefreesurf_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic01_prefreesurf.sh "
                          +results_path+" " 
                          +sub+" "
                          +params_struct[0]+" "             # T1w image
                          +params_struct[1]+" "             # T2w image
                          +params_struct[2]+" "             # AvgrdcSTRING
                          +params_struct[3]+" "             # Magnitudeimage 1
                          +params_struct[4]+" "             # Magnitudeimage 2
                          +params_struct[5]+" "             # Magnitudeimage 4D
                          +params_struct[6]+" "             # Phase image
                          +str(params_struct[7])+" "        # TE
                          +str(params_struct[8])+" "        # T1SampleSpacing
                          +str(params_struct[9])+" "        # T2SampleSpacing
                          +params_struct[10]                # UnwarpDirStruct
                          + "\n")

    batch_freesurf_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic02_freesurf.sh "
                          +results_path+" "
                          +sub+" "
                          +"\n")

    batch_postfreesurf_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic03_postfreesurf.sh "
                              +results_path+" "
                              +sub+" "
                              +"\n")



    ########################
    ###### fMRI Volume ##### 
    ########################

    params_fmri = fmri_scan_params(rawdata_remote_path, rawdata_local_path, sub)
    if params_fmri == False: continue # i.e. if Scanner Manufacturer == Philips/GE or unlisted

    # print to list
    batch_fmri_volume_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic04_fmri_volume.sh "+
                                  results_path+" "+
                                  sub+" "+
                                  params_fmri[0]+" "+        # UnwarpDirfMRI
                                  params_fmri[1]+" "+        # fMRI TImeseries
                                  params_fmri[2]+" "+        # DwellTime
                                  params_fmri[3]+" "+        # DistrotionCorrection
                                  params_struct[5]+" "+      # Magnitude 4D
                                  params_struct[6]+" "+      # Phaseimage
                                  params_struct[7]+" "+      # TE
                                  params_fmri[4]+"\n"        # FinalFMRIResolution
                                  )

    batch_fmri_surface_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic05_fmri_surface.sh "+
                              results_path+" "+
                              sub+" "+
                              params_fmri[4]+"\n")


    #######################
    ######### FIX ########
    #######################

    batch_icafix_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic06_icafix.sh "+
                              results_path+" "+
                              sub+" "+
                              resultsPath+"/"+sub+"/MNINonLinear/Results/Restingstate/Restingstate.nii.gz"+" "+
                              FIX_training_file+"\n")

    batch_postfix_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic07_postfix.sh "+
                              results_path+" "+
                              sub+"\n")


    #######################
    ####### MSM all #######
    #######################

    batch_msmall_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic08_msmall.sh "+
                              results_path+" "+
                              sub+"\n"+
                              "sleep 1m"+"\n")

    batch_dedriftresample_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic09_dedriftresample.sh "+
                              results_path+" "+
                              sub+"\n"+
                              "sleep 1m"+"\n")


    ########################
    ######### DWI ########## 
    ########################

    #get scan params and file names
    dwi_params = DWI_scan_params(rawdata_remote_path, rawdata_local_path, sub)

    batch_denoise_preproc_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic10_denoise_preproc.sh "+
                             results_path+" "+
                             sub+" "+
                             dwi_params[0]+" "+    # DWI image
                             dwi_params[1]+" "+    # DWI bvecs
                             dwi_params[2]+" "+    # DWI bval
                             dwi_params[3]+"\n")   # Phase-encoding dir

    batch_dwibiascorrect_dwi2mask_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic11_dwibiascorrect_dwi2mask.sh "+
                            results_path+" "+
                            sub +" "+
                            dwi_params[4]+" "+ # PE dir as 0x1x0 (ie. like ANTS format))
                            "True "+           # "Use Jacobian"
                            "True" + "\n")     # "Use bias field"

    #####batch_dwiintensitynorm_list comes later, outside loop.

    batch_T1w2dwi_5ttgen_dwi2response_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic13_T1w2dwi_5ttgen_dwi2response.sh "+
                            results_path+" "+
                            sub+"\n") 

    #batch_average_response_list.sh comes later, outside loop.
    
    batch_dwi_2fod_tckgen_sift2_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic15_dwi_2fod_tckgen_sift2.sh "+
                            results_path+" "+
                            sub+" "+
                            scratchPath+"\n")

    batch_create_diffusion_mask_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic16_create_diffusion_mask.sh "+
                            results_path+" "+
                            sub+"\n")

    batch_tck2connectome_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic17_tck2connectome.sh "+
                            results_path+" "+
                            sub+"\n")



    ########################
    ######### PET ##########
    ########################

    pet_params = PET_scan_params(rawdata_remote_path, rawdata_local_path, sub)

    batch_extract_PET_data_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic18_extract_PET_data.sh "+
                                sub+" "+
                                pet_params[0]+" "+
                                pet_params[1]+"\n")



    batch_extract_fmri_list.write("sbatch -o output_"+sub+".txt -e error_"+sub+".txt generic19_extract_fmri.sh "+
                                results_path+" "+
                                sub+" "+
                                "Restingstate"+"\n")

# close the sbatch submitters
batch_prefreesurf_list.close()
batch_freesurf_list.close()
batch_postfreesurf_list.close()
batch_fmri_volume_list.close()
batch_fmri_surface_list.close()
batch_icafix_list.close()
batch_postfix_list.close()
batch_msmall_list.close()
batch_dedriftresample_list.close()
batch_denoise_preproc_list.close()
batch_dwibiascorrect_dwi2mask_list.close()
batch_T1w2dwi_5ttgen_dwi2response_list.close()
batch_dwi_2fod_tckgen_sift2_list.close()
batch_create_diffusion_mask_list.close()
batch_tck2connectome_list.close()
batch_extract_PET_data_list.close()
batch_extract_fmri_list.close()

#batch12_dwiintensitynorm
batch_dwiintensitynorm_list = open(scripts_folder+"/batch12_dwiintensitynorm.sh","w") #only one line: uses all subs. write outside loop.
batch_dwiintensitynorm_list.write("sbatch -o output_dwiintensitynorm.txt -e error_dwiintensitynorm.txt generic12_dwiintensitynorm.sh"+" "+results_path)
batch_dwiintensitynorm_list.close()

#batch14_averageresponse
batch_averageresponse_list = open(scripts_folder+"/batch14_averageresponse.sh","w") #only one line: uses all subs. write outside loop.
batch_averageresponse_list.write("sbatch -o output_averageresponse.txt -e error_averageresponse.txt generic14_averageresponse.sh"+" "+results_path)
batch_averageresponse_list.close()

# Functions to retrieve image parameters

In [ ]:
def structural_scan_params(data_path, raw_data_path, sub):
    tmp_path=raw_data_path+"/"+sub  # local path of MRI data
    data_path=data_path+"/"+sub # path on BIH server
    
    # T1w and T2w Images
    T1_list=glob.glob(tmp_path+"/anat/"+"sub"+"*T1w*nii.gz")    
    
    # use FLAIR image instead of T2, cause T2 is actually T2 Star with bad resolution
    T2w_image=glob.glob(tmp_path+"/anat/"+"sub"+"*FLAIR*"+"*nii.gz")[0]
    
    if len(T2w_image) == 0:
        T2w_image = glob.glob(tmp_path+"/anat/"+"sub"+"*T2w*"+"*nii.gz")[0] #not T2STAR because some of them are actually fieldmaps (ADNI mislabelling)
    
    # get Scanner Type
    data = json.loads(open(T1_list[0][:-6]+"json").read()) ##Manufacturer = json.loads(open(T1_list[0][:-6]+"json").read())['Manufacturer'] 
    if "Manufacturer" in data:
        Manufacturer = data["Manufacturer"]    
    elif "ManufacturersModelName" in data:
        if data["ManufacturersModelName"] == "Skyra_fit":
            Manufacturer = "Siemens"
        else:
            Manufacturer = " "
    else:
        Manufacturer = " "
        
    if Manufacturer=='Philips' or Manufacturer=="GE" or Manufacturer==" ": return False
    
    Model = json.loads(open(T1_list[0][:-6]+"json").read())['ManufacturersModelName']
    
    # when more then one T1 image exist
    if len(T1_list)>1:
        # if same date, take the one WITH gradient distortion correction 
        T1_json_data = open(T1_list[0][:-6]+"json").read()
        data         = json.loads(T1_json_data)
        if len(data['ImageType'])<3:
            DC           = data['ImageType'][3]
            if DC=="NORM":
                T1w_image=T1_list[0]
            else:
                T1w_image=T1_list[1]
        else: # if T1 images differ on other feature than distortion correction use the first
            T1w_image=T1_list[0]
    else:
        T1w_image=T1_list[0]
            
                                           
#######
####### Siemens
#######
    if Manufacturer=="Siemens":
        
        AvgrdcSTRING="SiemensFieldMap"
        
        # Magnitude and Phase image
        MagnitudeImage1 = glob.glob(tmp_path+"/fmap/"+"sub"+"*_magnitude1*"+".nii.gz")[0]
        MagnitudeImage2 = glob.glob(tmp_path+"/fmap/"+"sub"+"*_magnitude2*"+".nii.gz")[0]
        PhaseImage = glob.glob(tmp_path+"/fmap/"+"sub"+"*phasediff*"+".nii.gz")[0]
        
        data = json.loads(open(MagnitudeImage1[:-7]+".json").read())
        EchoTime1 = data['EchoTime']
        data = json.loads(open(MagnitudeImage2[:-7]+".json").read())
        EchoTime2 = data['EchoTime']
                    
        # for fslmerge create a name:
        MagnitudeImage4D="4DMagnitudeImage.nii.gz"
                        
        # DwellTime for the fieldmap
        TE=format(abs(EchoTime1-EchoTime2)*1000,'.2f') # convert to ms

        # T1SampleSpacing and T2SampleSpacing: 
        # DWT = 1/ReceiverBandWidth = 1 / PixelBandWidth * NumberFrequencyEncondingSteps / ParallelReductionFactorInPlane
        data = json.loads(open(T1w_image[:-7]+".json").read())
        if 'DwellTime' in data:
            T1SampleSpacing=format(data['DwellTime'], '.8f')
        elif 'ParallelReductionFactorInPlane' in data :
            T1SampleSpacing= (1 / (data['PixelBandwidth'] * data['BaseResolution'])) / data['ParallelReductionFactorInPlane']
        elif 'BaseResolution' in data : 
            T1SampleSpacing= 1 / (data['PixelBandwidth'] * data['BaseResolution'])
        elif 'ReconMatrixPE' in data :
            T1SampleSpacing= 1 / (data['PixelBandwidth'] * data['ReconMatrixPE'])
        else :
            print("problem for sub",sub," with T1SampleSpacing.")
        
        data = json.loads(open(T2w_image[:-7]+".json").read())
        if 'DwellTime' in data:
            T2SampleSpacing=format(data['DwellTime'], '.8f')
        elif 'ParallelReductionFactorInPlane' in data :
            T2SampleSpacing= (1 / (data['PixelBandwidth'] * data['ReconMatrixPE'])) / data['ParallelReductionFactorInPlane']
        else : 
            T2SampleSpacing= 1 / (data['PixelBandwidth'] * data['ReconMatrixPE'])        
                
        # Unwarp direction for structural scans
        UnwarpDirStruct_name=data['InPlanePhaseEncodingDirectionDICOM']
        #print(UnwarpDirStruct_name)
        if UnwarpDirStruct_name == "ROW":
            UnwarpDirStruct="y" # in json "InPlanePhaseEncodingDirectionDICOM": "ROW",
        else:
            print("Unwarp dir not ROW")
            
        return (data_path+"/anat/"+os.path.basename(T1w_image), 
            data_path+"/anat/"+os.path.basename(T2w_image),
            AvgrdcSTRING,
            data_path+"/fmap/"+os.path.basename(MagnitudeImage1),
            data_path+"/fmap/"+os.path.basename(MagnitudeImage2),
            data_path+"/fmap/"+MagnitudeImage4D,
            data_path+"/fmap/"+os.path.basename(PhaseImage),
            str(TE),
            str(T1SampleSpacing),
            str(T2SampleSpacing),
            UnwarpDirStruct)
            

#######
####### GE
#######
    
    elif Manufacturer=="GE": 
        # ADNI GE Scanners don't provide any kind of fieldmaps
        # so no distortion correction is possible
        AvgrdcSTRING="NONE"
        MagnitudeImage1="NONE"
        MagnitudeImage2="NONE"
        MagnitudeImage4D="NONE"
        PhaseImage="NONE"
        TE="NONE"
        T1SampleSpacing="NONE"
        T2SampleSpacing="NONE"
        UnwarpDirStruct="NONE"
        
        return (data_path+os.path.basename(T1w_image), 
            data_path+os.path.basename(T2w_image),
            AvgrdcSTRING,
            MagnitudeImage1,
            MagnitudeImage2,
            MagnitudeImage4D,
            PhaseImage,
            TE,
            T1SampleSpacing,
            T2SampleSpacing,
            UnwarpDirStruct)        
        
#######
####### Philips
#######
    
    elif Manufacturer=="Philips":
        ###
        AvgrdcSTRING="SiemensFieldMap"
        
        MagnitudeImage1="NONE"
        MagnitudeImage2="NONE"
        MagnitudeImage4D="NONE"
        PhaseImage1="NONE"
        PhaseImage2="NONE"
        TE="NONE"

        
        # T1SampleSpacing and T2SampleSpacing: 
        # DWT = 1/ReceiverBandWidth = 1 / PxielBandWidth * NumberFrequencyEncondingSteps / ParallelReductionFactorInPlane
        data = json.loads(open(T1w_image[:-7]+".json").read())
        if 'DwellTime' in data:
            T1SampleSpacing=format(data['DwellTime'], '.8f')
        elif 'ParallelReductionFactorInPlane' in data :
            T1SampleSpacing= (1 / (data['PixelBandwidth'] * data['BaseResolution'])) / data['ParallelReductionFactorInPlane']
        else : 
            T1SampleSpacing= 1 / (data['PixelBandwidth'] * data['BaseResolution'])
        
        data = json.loads(open(T1w_image[:-7]+".json").read())
        if 'DwellTime' in data:
            T1SampleSpacing=format(data['DwellTime'], '.8f')
        elif 'ParallelReductionFactorInPlane' in data :
            T1SampleSpacing= (1 / (data['PixelBandwidth'] * data['BaseResolution'])) / data['ParallelReductionFactorInPlane']
        else : 
            T1SampleSpacing= 1 / (data['PixelBandwidth'] * data['BaseResolution'])
            
        T2samplespacing.append(T2SampleSpacing)
        
        UnwarpDirStruct_name=data['InPlanePhaseEncodingDirectionDICOM']
        print(UnwarpDirStruct_name)
        
        if UnwarpDirStruct_name == "ROW":
            UnwarpDirStruct="y" # in json "InPlanePhaseEncodingDirectionDICOM": "ROW",
        else:
            print("Unwarp dir not ROW")
    
        return (data_path + "/anat/" + os.path.basename(T1w_image), 
                data_path + "/anat/" + os.path.basename(T2w_image),
                AvgrdcSTRING,
                data_path + "/fmap/" + os.path.basename(MagnitudeImage1),
                data_path + "/fmap/" +os.path.basename(MagnitudeImage2),
                data_path + "/fmap/" +MagnitudeImage4D,
                data_path + "/fmap/" +os.path.basename(PhaseImage),
                str(TE),
                str(T1SampleSpacing),
                str(T2SampleSpacing),
                UnwarpDirStruct)
            
    

In [ ]:
def fmri_scan_params(data_path, raw_data_path, sub):
    tmp_path = raw_data_path+"/"+sub  # local path of MRI data
    data_path = data_path+"/"+sub # path on BIH server
    
    # fMRI image
    fMRITimeSeries=glob.glob(tmp_path+"/func/"+"sub"+"*bold*"+"*nii.gz")[0]
    data = json.loads(open(fMRITimeSeries[:-7]+".json").read())
    #Image Resolution
    FinalFMRIResolution=data['SliceThickness']    
    
    #Manufacturer
    if "Manufacturer" in data:
        Manufacturer = data["Manufacturer"]    
    elif "ManufacturersModelName" in data:
        if data["ManufacturersModelName"] == "Skyra_fit":
            Manufacturer = "Siemens"
        else:
            Manufacturer = " "
    else:
        Manufacturer = " "
    
    if Manufacturer=='Philips' or Manufacturer=="GE" or Manufacturer==" ": return False

    if Manufacturer=="Siemens":
        DistortionCorrection="SiemensFieldMap"
        PEdir = data['PhaseEncodingDirection']

        def switch_PEdir(argument):
            switcher = {
                "i" : "x",
                "i-": "-x",
                "j" : "y",
                "j-": "-y",
                "k" : "z",
                "k-": "-z",
            }
            return switcher.get(argument, "Invalid direction")

        UnwarpDirfMRI=switch_PEdir(PEdir)

        # DwellTime
        # BPPPE = data['BandwidthPerPixelPhaseEncode']
        # n_PE_samples= data['AcquisitionMatrixPE']
        # DwellTime=1/(BPPPE*n_PE_samples)/ParallelReductionFactor (or Accelaration) #--> equals 'EffectiveEchoSpacing' in .json
        DwellTime = data['EffectiveEchoSpacing']
        

    elif Manufacturer == "GE":
        UnwarpDirfMRI = "NONE"
        DwellTime     = "NONE"
        DistortionCorrection = "NONE"


    elif Manufacturer =="Philips":
        print("Philips FMRI NOT YET INCLUDED")
        print("Philips FMRI NOT YET INCLUDED")
        print("Philips FMRI NOT YET INCLUDED")
        print("Philips FMRI NOT YET INCLUDED")
        UnwarpDirfMRI = "NONE"
        DwellTime     = "NONE"
        DistortionCorrection = "NONE"
        
    return (UnwarpDirfMRI, 
            data_path + "/func/" + os.path.basename(fMRITimeSeries),
            str(DwellTime),
            DistortionCorrection,
            str(FinalFMRIResolution))
            

In [ ]:
def DWI_scan_params(data_path, raw_data_path, sub):
    
    tmp_path = raw_data_path+"/"+sub  # local path of MRI data
    data_path = data_path+"/"+sub # path on BIH server
    
    # dwi image
    # some subjects have different DTI files, use the one we find bvec and bvals for
    json_list = glob.glob(tmp_path+"/dwi/"+"sub"+"*dwi*""*json")
    for j in json_list:
        json_file = j
        bvecs = json_file[0:-5]+".bvec"
        bvals = json_file[0:-5]+".bval"
        dwi_image = json_file[0:-5]+".nii.gz"

        if ((not bvecs) | (not bvals) | (not dwi_image)):
            continue
        else:
            break        
    
    #  dwidenoise dwipreproc
    data = json.loads(open(dwi_image[:-7]+".json").read())
    def switch_PEdir(argument):
        switcher = {
            "i" : "RL",
            "i-": "LR",
            "j" : "PA",
            "j-": "AP",
            "k" : "IS",
            "k-": "SI",
        }
        return switcher.get(argument, "Invalid direction")

    UnwarpDirDWI=switch_PEdir(data['PhaseEncodingDirection'])
    
    EffectiveEchoSpacing = data['EffectiveEchoSpacing']
    
    if UnwarpDirDWI=="RL" or UnwarpDirDWI=="LR":
        UnwarpDirDWI_ants = "1x0x0"
    elif UnwarpDirDWI=="PA" or UnwarpDirDWI=="AP":
        UnwarpDirDWI_ants = "0x1x0"
    elif UnwarpDirDWI=="IS" or UnwarpDirDWI=="SI":
        UnwarpDirDWI_ants = "0x0x1"
    
    
    return (data_path+"/dwi/"+os.path.basename(dwi_image), 
            data_path+"/dwi/"+os.path.basename(bvecs), 
            data_path+"/dwi/"+os.path.basename(bvals),
            UnwarpDirDWI,
            UnwarpDirDWI_ants) 

In [ ]:
def PET_scan_params(data_path, raw_data_path, sub):
    tmp_path=raw_data_path+"/"+sub  # local path of MRI data
    data_path=data_path+"/"+sub # path on BIH server
    
    AV1451_image=glob.glob(tmp_path+"/pet/"+"sub"+"*AV1451*"+"*nii.gz")[0]
    AV45_image=glob.glob(tmp_path+"/pet/"+"sub"+"*AV45*"+"*nii.gz")[0]
    
    return(data_path+"/pet/"+os.path.basename(AV1451_image),
           data_path+"/pet/"+os.path.basename(AV45_image))

